In [0]:
%run ./loadData

In [0]:
from pyspark.sql.types import BooleanType, DoubleType, IntegerType, StringType, StructField, StructType
from pyspark.sql.functions import expr, lit, regexp_replace, when
dbutils.fs.ls("/mnt/worldbankdata")

Out[3]: [FileInfo(path='dbfs:/mnt/worldbankdata/AccessToElectricityPopulationPercentage.csv', name='AccessToElectricityPopulationPercentage.csv', size=81970),
 FileInfo(path='dbfs:/mnt/worldbankdata/AgriculturalLandPercentageLandArea.csv', name='AgriculturalLandPercentageLandArea.csv', size=181990),
 FileInfo(path='dbfs:/mnt/worldbankdata/AgricultureForestryFishingValueAddedPercentageGDP.csv', name='AgricultureForestryFishingValueAddedPercentageGDP.csv', size=151612),
 FileInfo(path='dbfs:/mnt/worldbankdata/ArableLandPercentageLandArea.csv', name='ArableLandPercentageLandArea.csv', size=179265),
 FileInfo(path='dbfs:/mnt/worldbankdata/CO2EmissionsMetricTonsPerCapita.csv', name='CO2EmissionsMetricTonsPerCapita.csv', size=180623),
 FileInfo(path='dbfs:/mnt/worldbankdata/DiabetesPrevalencePopulation20To79Percentage.csv', name='DiabetesPrevalencePopulation20To79Percentage.csv', size=40543),
 FileInfo(path='dbfs:/mnt/worldbankdata/ElectricPowerConsumptionKWhPerCapita.csv', name='ElectricPowerConsumptionKWhPerCapita.csv', size=122763),
 FileInfo(path='dbfs:/mnt/worldbankdata/FertilityRateTotal.csv', name='FertilityRateTotal.csv', size=123035),
 FileInfo(path='dbfs:/mnt/worldbankdata/ForestAreaPercentageLandArea.csv', name='ForestAreaPercentageLandArea.csv', size=118417),
 FileInfo(path='dbfs:/mnt/worldbankdata/GDPGrowthAnnualPercentage.csv', name='GDPGrowthAnnualPercentage.csv', size=164850),
 FileInfo(path='dbfs:/mnt/worldbankdata/GDPUSD.csv', name='GDPUSD.csv', size=166530),
 FileInfo(path='dbfs:/mnt/worldbankdata/GovernmentExpenditureEducationPercentageGDP.csv', name='GovernmentExpenditureEducationPercentageGDP.csv', size=98651),
 FileInfo(path='dbfs:/mnt/worldbankdata/IncomeShareHighest20Percent.csv', name='IncomeShareHighest20Percent.csv', size=40868),
 FileInfo(path='dbfs:/mnt/worldbankdata/IncomeShareLowest20Percent.csv', name='IncomeShareLowest20Percent.csv', size=38925),
 FileInfo(path='dbfs:/mnt/worldbankdata/IndustryValueAddedPercentageGDP.csv', name='IndustryValueAddedPercentageGDP.csv', size=150221),
 FileInfo(path='dbfs:/mnt/worldbankdata/InflationConsumerPricesAnnualPercentage.csv', name='InflationConsumerPricesAnnualPercentage.csv', size=147902),
 FileInfo(path='dbfs:/mnt/worldbankdata/LifeExpectancyAtBirth.csv', name='LifeExpectancyAtBirth.csv', size=152304),
 FileInfo(path='dbfs:/mnt/worldbankdata/LiteracyRateAdultPercentage15AndAbove.csv', name='LiteracyRateAdultPercentage15AndAbove.csv', size=70056),
 FileInfo(path='dbfs:/mnt/worldbankdata/LiteracyRateFemalePercentage15AndAbove.csv', name='LiteracyRateFemalePercentage15AndAbove.csv', size=71278),
 FileInfo(path='dbfs:/mnt/worldbankdata/LiteracyRateMalePercentage15AndAbove.csv', name='LiteracyRateMalePercentage15AndAbove.csv', size=70266),
 FileInfo(path='dbfs:/mnt/worldbankdata/MortalityRateUnder5Per1000Births.csv', name='MortalityRateUnder5Per1000Births.csv', size=92950),
 FileInfo(path='dbfs:/mnt/worldbankdata/MortalityRoadTrafficPer100000Population.csv', name='MortalityRoadTrafficPer100000Population.csv', size=66843),
 FileInfo(path='dbfs:/mnt/worldbankdata/NetMigration.csv', name='NetMigration.csv', size=45770),
 FileInfo(path='dbfs:/mnt/worldbankdata/Population0To14TotalPercentage.csv', name='Population0To14TotalPercentage.csv', size=198885),
 FileInfo(path='dbfs:/mnt/worldbankdata/Population15To64TotalPercentage.csv', name='Population15To64TotalPercentage.csv', size=199167),
 FileInfo(path='dbfs:/mnt/worldbankdata/PopulationExcessivePollutionExposureTotalPercentage.csv', name='PopulationExcessivePollutionExposureTotalPercentage.csv', size=68159),
 FileInfo(path='dbfs:/mnt/worldbankdata/PopulationGrowthAnnualPercentage.csv', name='PopulationGrowthAnnualPercentage.csv', size=206114),
 FileInfo(path='dbfs:/mnt/worldbankdata/PopulationLivingSlumsUrbanPopulationPercentage.csv', name='PopulationLivingSlumsUrbanPopulationPercentage.csv', size=48100),
 FileInfo(path='dbfs:/mnt/worldbankdata/PopulationTotal.csv', name='PopulationTotal.csv', size=147884),
 FileInfo(p

/mnt/worldbankdata has been unmounted.
Out[2]: True

In [0]:
# Get the Life Expectancy CSV and show some sample data.
dfLifeExp = spark.read.option("header", "true").csv(lifeExpCsv)
dfLifeExp.select(dfLifeExp.columns[0:5]).show()

+--------------------+------------+--------------------+--------------+-----------+
 Country Name|Country Code| Indicator Name|Indicator Code| 1960|
+--------------------+------------+--------------------+--------------+-----------+
 Aruba| ABW|Life expectancy a...|SP.DYN.LE00.IN| 65.662|
Africa Eastern an...| AFE|Life expectancy a...|SP.DYN.LE00.IN|42.71605253|
 Afghanistan| AFG|Life expectancy a...|SP.DYN.LE00.IN| 32.446|
Africa Western an...| AFW|Life expectancy a...|SP.DYN.LE00.IN|37.20538029|
 Angola| AGO|Life expectancy a...|SP.DYN.LE00.IN| 37.524|
 Albania| ALB|Life expectancy a...|SP.DYN.LE00.IN| 62.283|
 Andorra| AND|Life expectancy a...|SP.DYN.LE00.IN| null|
 Arab World| ARB|Life expectancy a...|SP.DYN.LE00.IN|46.54690911|
United Arab Emirates| ARE|Life expectancy a...|SP.DYN.LE00.IN| 51.537|
 Argentina| ARG|Life expectancy a...|SP.DYN.LE00.IN| 65.055|
 Armenia| ARM|Life expectancy a...|SP.DYN.LE00.IN| 65.972|
 American Samoa| ASM|Life expectancy a...|SP.DYN.LE00.IN| null|
 Antigua and Barbuda| ATG|Life expectancy a...|SP.DYN.LE00.IN| 61.968|
 Australia| AUS|Life expectancy a...|SP.DYN.LE00.IN|70.81707317|
 Austria| AUT|Life expectancy a...|SP.DYN.LE00.IN|68.58560976|
 Azerbaijan| AZE|Life expectancy a...|SP.DYN.LE00.IN| 61.034|
 Burundi| BDI|Life expectancy a...|SP.DYN.LE00.IN| 41.281|
 Belgium| BEL|Life expectancy a...|SP.DYN.LE00.IN|69.70195122|
 Benin| BEN|Life expectancy a...|SP.DYN.LE00.IN| 37.271|
 Burkina Faso| BFA|Life expectancy a...|SP.DYN.LE00.IN| 34.432|
+--------------------+------------+--------------------+--------------+-----------+
only showing top 20 rows

In [0]:
# Create the first dimension table for countries.
# Some of the entries are regions rather than countries - add 2 columns to table to reflect this.
from pyspark.sql.functions import lit
dimCountrySchema = StructType([ \
                               StructField("regionCode", StringType(), True), \
                               StructField("countryName", StringType(), True), \
                               StructField("isCountry", BooleanType(), True), \
                               StructField("isRegion", BooleanType(), True), \
                              ])
countriesSet = {"ABW", "AFG", "AGO", "ALB", "AND", "ARE", "ARG", "ARM", "ASM", "ATG",
                "AUS", "AUT", "AZE", "BDI", "BEL", "BEN", "BFA", "BGD", "BGR", "BHR",
                "BHS", "BIH", "BLR", "BLZ", "BMU", "BOL", "BRA", "BRB", "BRN", "BTN", 
                "BWA", "CAF", "CAN", "CHE", "CHI", "CHL", "CHN", "CIV", "CMR", "COD",
                "COG", "COL", "COM", "CPV", "CRI", "CUB", "CUW", "CYM", "CYP", "CZE",
                "DEU", "DJI", "DMA", "DNK", "DOM", "DZA", "ECU", "EGY", "ERI", "ESP",
                "EST", "ETH", "FIN", "FJI", "FRA", "FRO", "FSM", "GAB", "GBR", "GEO",
                "GHA", "GIB", "GIN", "GMB", "GNB", "GNQ", "GRC", "GRD", "GRL", "GTM",
                "GUM", "GUY", "HKG", "HND", "HRV", "HTI", "HUN", "IDN", "IMN", "IND",
                "IRL", "IRN", "IRQ", "ISL", "ISR", "ITA", "JAM", "JOR", "JPN", "KAZ",
                "KEN", "KGZ", "KHM", "KIR", "KNA", "KOR", "KWT", "LAO", "LBN", "LBR",
                "LBY", "LCA", "LIE", "LKA", "LSO", "LTU", "LUX", "LVA", "MAC", "MAF",
                "MAR", "MCO", "MDA", "MDG", "MDV", "MEX", "MHL", "MKD", "MLI", "MLT",
                "MMR", "MNE", "MNG", "MNP", "MOZ", "MRT", "MUS", "MWI", "MYS", "NAM",
                "NCL", "NER", "NGA", "NIC", "NLD", "NOR", "NPL", "NRU", "NZL", "OMN",
                "PAK", "PAN", "PER", "PHL", "PLW", "PNG", "POL", "PRI", "PRK", "PRT",
                "PRY", "PSE", "PYF", "QAT", "ROU", "RUS", "RWA", "SAU", "SDN", "SEN",
                "SGP", "SLB", "SLE", "SLV", "SMR", "SOM", "SRB", "SSD", "STP", "SUR",
                "SVN", "SVN", "SWE", "SWZ", "SXM", "SYC", "SYC", "TCA", "TCD", "TGO",
                "THA", "TJK", "TKM", "TLS", "TON", "TTO", "TUN", "TUR", "TUV", "TZA",
                "UGA", "UKR", "URY", "USA", "UZB", "VCT", "VEN", "VGB", "VIR", "VNM",
                "VUT", "WSM", "XKX", "YEM", "ZAF", "ZMB", "ZWE"}

regionSet = {"AFE", "AFW", "ARB", "CEB", "CSS", "EAS", "ECS", "EMU", "EUU", "LCN",
             "MEA", "NAC", "SAS", "SSF"}

dimCountry = dfLifeExp.select(col("Country Code").alias("regionCode"), col("Country Name").alias("countryName"))
dimCountry = dimCountry.withColumn("isCountry", when(dimCountry.regionCode.isin(countriesSet), True).otherwise(None)).withColumn("isRegion", when(dimCountry.regionCode.isin(regionSet), True).otherwise(None))
display(dimCountry)

regionCode,countryName,isCountry,isRegion
ABW,Aruba,true,null
AFE,Africa Eastern and Southern,null,true
AFG,Afghanistan,true,null
AFW,Africa Western and Central,null,true
AGO,Angola,true,null
ALB,Albania,true,null
AND,Andorra,true,null
ARB,Arab World,null,true
ARE,United Arab Emirates,true,null
ARG,Argentina,true,null


In [0]:
# Create the second dimension table for indicator codes.
dimIndicatorSchema = StructType([ \
                                 StructField("indicatorCode", StringType(), True), \
                                 StructField("indicatorName", StringType(), True), \
                                ])
dimIndicatorData = []

for csv in csvList:
    df = spark.read.option("header", "true").csv(csv)
    indicatorCode = df.select(col("Indicator Code")).collect()[0][0]
    indicatorName = df.select(col("Indicator Name")).collect()[0][0]
    tuple = (indicatorCode, indicatorName)
    dimIndicatorData.append(tuple)

dimIndicator = spark.createDataFrame(data=dimIndicatorData, schema=dimIndicatorSchema)
display(dimIndicator)

indicatorCode,indicatorName
EG.ELC.ACCS.ZS,Access to electricity (% of population)
AG.LND.AGRI.ZS,Agricultural land (% of land area)
NV.AGR.TOTL.ZS,"Agriculture, forestry, and fishing, value added (% of GDP)"
AG.LND.ARBL.ZS,Arable land (% of land area)
EN.ATM.CO2E.PC,CO2 emissions (metric tons per capita)
SH.STA.DIAB.ZS,Diabetes prevalence (% of population ages 20 to 79)
EG.USE.ELEC.KH.PC,Electric power consumption (kWh per capita)
SP.DYN.TFRT.IN,"Fertility rate, total (births per woman)"
AG.LND.FRST.ZS,Forest area (% of land area)
NY.GDP.MKTP.KD.ZG,GDP growth (annual %)


In [0]:
# Create the third dimension table for dates
dimDateSchema = StructType([ \
                            StructField("dateYear", IntegerType(), True), \
                            StructField("dateDecade", StringType(), True), \
                           ])
dimDateData = []

for year in range(1960, 2021):
    if 1960 <= year < 1970:
        tuple = (year, "1960s")
    elif 1970 <= year < 1980:
        tuple = (year, "1970s")
    elif 1980 <= year < 1990:
        tuple = (year, "1980s")
    elif 1990 <= year < 2000:
        tuple = (year, "1990s")
    elif 2000 <= year < 2010:
        tuple = (year, "2000s")
    elif 2010 <= year < 2020:
        tuple = (year, "2010s")
    else:
        tuple = (year, "2020s")
    dimDateData.append(tuple)

dimDate = spark.createDataFrame(data=dimDateData, schema=dimDateSchema)
display(dimDate)

dateYear,dateDecade
1960,1960s
1961,1960s
1962,1960s
1963,1960s
1964,1960s
1965,1960s
1966,1960s
1967,1960s
1968,1960s
1969,1960s


In [0]:
# Next, create the fact table.
from pyspark.sql.types import BooleanType, DoubleType, IntegerType, StringType, StructField, StructType
factSchema = StructType([ \
                            StructField("regionCode", StringType(), True), \
                            StructField("indicatorCode", StringType(), True), \
                            StructField("dateYear", IntegerType(), True), \
                            StructField("value", DoubleType(), True), \
                           ])
factData = []

for csv in csvList:
    df = spark.read.option("header", "true").csv(csv)
    numCodes = df.select(col("Country Code")).count()
    indicatorCode = df.select(col("Indicator Code")).collect()[0][0]
    regionColumn = df.select(col("Country Code")).collect()
    for year in range(1960, 2021):
        yearStr = str(year)
        yearColumn = df.select(col(yearStr)).collect()
        for counter in range(0, numCodes):
            regionCode = regionColumn[counter].asDict()['Country Code']
            value = yearColumn[counter].asDict()[yearStr]
            if value is not None:
                valueFloat = float(value)
                tuple = (regionCode, indicatorCode, year, valueFloat)
            else:
                tuple = (regionCode, indicatorCode, year, value)
            factData.append(tuple)

factTable = spark.createDataFrame(data=factData, schema=factSchema)
factTable.show()

+----------+--------------+--------+-----+
regionCode| indicatorCode|dateYear|value|
+----------+--------------+--------+-----+
 ABW|EG.ELC.ACCS.ZS| 1960| null|
 AFE|EG.ELC.ACCS.ZS| 1960| null|
 AFG|EG.ELC.ACCS.ZS| 1960| null|
 AFW|EG.ELC.ACCS.ZS| 1960| null|
 AGO|EG.ELC.ACCS.ZS| 1960| null|
 ALB|EG.ELC.ACCS.ZS| 1960| null|
 AND|EG.ELC.ACCS.ZS| 1960| null|
 ARB|EG.ELC.ACCS.ZS| 1960| null|
 ARE|EG.ELC.ACCS.ZS| 1960| null|
 ARG|EG.ELC.ACCS.ZS| 1960| null|
 ARM|EG.ELC.ACCS.ZS| 1960| null|
 ASM|EG.ELC.ACCS.ZS| 1960| null|
 ATG|EG.ELC.ACCS.ZS| 1960| null|
 AUS|EG.ELC.ACCS.ZS| 1960| null|
 AUT|EG.ELC.ACCS.ZS| 1960| null|
 AZE|EG.ELC.ACCS.ZS| 1960| null|
 BDI|EG.ELC.ACCS.ZS| 1960| null|
 BEL|EG.ELC.ACCS.ZS| 1960| null|
 BEN|EG.ELC.ACCS.ZS| 1960| null|
 BFA|EG.ELC.ACCS.ZS| 1960| null|
+----------+--------------+--------+-----+
only showing top 20 rows

In [0]:
# Try creating the fact table by unpivoting. to see if we get better performance
factSchema = StructType([ \
                            StructField("regionCode", StringType(), True), \
                            StructField("indicatorCode", StringType(), True), \
                            StructField("dateYear", IntegerType(), True), \
                            StructField("value", DoubleType(), True), \
                           ])

factTable2 = spark.createDataFrame(data=[], schema=factSchema)
for csv in csvList:
    ingestedDf = spark.read.option("header", "true").csv(csv)
    factDf = ingestedDf.select(col("Country Code"), col("Indicator Code"), lit(ingestedDf.columns[4]).alias("dateYear"),ingestedDf[ingestedDf.columns[4]].alias("value"))
    for counter in ingestedDf.columns[5:]:
        factDf = factDf.union(ingestedDf.select(col("Country Code"), col("Indicator Code"), lit(counter), ingestedDf[counter]))
    factDf = factDf.withColumnRenamed("Country Code", "regionCode").withColumnRenamed("Indicator Code", "indicatorCode")
    factTable2 = factTable.union(factDf)

factTable2 = factTable2.withColumn("value", col("value").cast('double')).withColumn("dateYear", col("dateYear").cast('integer'))
display(factTable2)

regionCode,indicatorCode,dateYear,value
ABW,EG.ELC.ACCS.ZS,1960,null
AFE,EG.ELC.ACCS.ZS,1960,null
AFG,EG.ELC.ACCS.ZS,1960,null
AFW,EG.ELC.ACCS.ZS,1960,null
AGO,EG.ELC.ACCS.ZS,1960,null
ALB,EG.ELC.ACCS.ZS,1960,null
AND,EG.ELC.ACCS.ZS,1960,null
ARB,EG.ELC.ACCS.ZS,1960,null
ARE,EG.ELC.ACCS.ZS,1960,null
ARG,EG.ELC.ACCS.ZS,1960,null


In [0]:
# Count the number of rows in the tables
dimCountry.count()
dimIndicator.count()
dimDate.count()
factTable.count()
factTable2.count()

Out[10]: 567910

In [0]:
# Finally, save the tables! Because factTable2 is significantly faster than factTable1, we pick factTable2 as our primary Fact table.
dimCountry.write.mode("overwrite").saveAsTable("DimensionTableCountryCode")
dimIndicator.write.mode("overwrite").saveAsTable("DimensionTableIndicatorCode")
dimDate.write.mode("overwrite").saveAsTable("DimensionTableDate")
factTable.write.mode("overwrite").saveAsTable("FactTableBackup")
factTable2.write.mode("overwrite").saveAsTable("FactTable")